### Imports  & Parameters

In [8]:
%load_ext autoreload
%autoreload 2

import os, sys
# ugly workaround for imports
sys.path.append(os.path.split(os.getcwd())[0])

import numpy as np
import scipy as sp
import matplotlib.pyplot as pl

from fit import *
from utils.prf import *

In [2]:
# parameters of analysis
extent = [-5, 5]
stim_radius = 5.0
n_pix = 41
rsq_threshold = 0.5

# make mask based on these parameters
mask = create_circular_mask(n_pix,n_pix,radius=stim_radius)

In [3]:
# settings that have to do with the data and experiment
nr_prf_parameters = 8
TR = 0.945
screen_distance = 225
screen_width = 69
nr_TRs = 462
timepoints = np.arange(nr_TRs) * TR
n_folds = 6

### Get Data

In [4]:
(prf_cv_fold_data, rfs, linear_predictor, 
 all_residuals_css, all_residual_covariance_css, 
 stimulus_covariance_WW, test_data) = setup_data_from_h5(
                        data_file = '../data/V1.h5', 
                        n_pix=n_pix, 
                        extent=extent, 
                        stim_radius=stim_radius, 
                        screen_distance=screen_distance, 
                        screen_width=screen_width, 
                        rsq_threshold=0.5,
                        TR=TR,
                        cv_fold=0,
                        n_folds=n_folds)

data file found, returning local file ../data/V1.h5
Taking data corresponding to ['sub-012_task-mapper_acq-multiband_run-1_bold_B0_mcf_sg_psc', 'sub-012_task-mapper_acq-multiband_run-2_bold_B0_mcf_sg_psc', 'sub-012_task-mapper_acq-multiband_run-3_bold_B0_mcf_sg_psc', 'sub-012_task-mapper_acq-multiband_run-4_bold_B0_mcf_sg_psc', 'sub-012_task-mapper_acq-multiband_run-5_bold_B0_mcf_sg_psc', 'sub-012_task-mapper_acq-multiband_run-6_bold_B0_mcf_sg_psc'] from group /psc/V1.thresh_vol_dil.nii.gz
Taking data corresponding to ['sub-012_task-mapper_acq-multiband_run-1_bold_B0_mcf_sg_psc_av_loo', 'sub-012_task-mapper_acq-multiband_run-2_bold_B0_mcf_sg_psc_av_loo', 'sub-012_task-mapper_acq-multiband_run-3_bold_B0_mcf_sg_psc_av_loo', 'sub-012_task-mapper_acq-multiband_run-4_bold_B0_mcf_sg_psc_av_loo', 'sub-012_task-mapper_acq-multiband_run-5_bold_B0_mcf_sg_psc_av_loo', 'sub-012_task-mapper_acq-multiband_run-6_bold_B0_mcf_sg_psc_av_loo'] from group /loo/V1.thresh_vol_dil.nii.gz
Taking data correspo

In [9]:
# estimate the covariance structure, which outputs all parameters
(estimated_tau_matrix, estimated_rho, 
 estimated_sigma, model_omega, 
 model_omega_inv, logdet) = fit_model_omega(
                                        all_residual_covariance_css, 
                                        stimulus_covariance_WW,
                                        verbose=1)

NameError: name 'all_residual_covariance_css' is not defined

In [ ]:
# set up result arrays:
dm_pixel_logl_ratio = np.zeros((n_pix,n_pix,test_data.shape[1]))
decoded_image = np.zeros((n_pix,n_pix,test_data.shape[1]))  

# and loop across timepoints
for t, bold in enumerate(test_data.T):
    dm_pixel_logl_ratio[:,:,t] = firstpass_decoder_independent_pixels(bold, 
                                     logdet,
                                     omega_inv,
                                     linear_predictor,
                                     mask)
    decoded_image[:,:,t] = maximize_loglikelihood(
                            starting_value=dm_pixel_logl_ratio[:,:,t],
                            bold,
                            logdet,
                            omega_inv,
                            linear_predictor, 
                            mask)